# Markov Chain
1. 문장을 형태소 분할
2. 단어 전후를 사전 등록
3. 사전 이용 문장 생성

> 그 는 고양이
>
> 는 고양이 를
>
> 고양이 를 좋아
>
> 를 좋아 합니다

## 문제점

> 개 도 닷새 가 되면 주인을 안다.
>
> 기르던 개 에게 다리 가 물렸다.
>
> 닭 쫓던 개 지붕 쳐다 보듯 한다.
>
> 똥 묻은 개 가 겨 묻은 개 나무란다.

- 개로 문장을 시작 했을때 개 뒤에 연결되는 도/에게/지붕/가
- 예를 들어 가 로 연결되었을때 가에 연결되는 되면/겨
- 개 가 되면 주인 을 안다. 같이 이상한 문장이 생성됨 
- 문제점이 있기 때문에 LSTM으로 확률적인 보완

In [1]:
#import codecs
from bs4 import BeautifulSoup
from konlpy.tag import Twitter
import urllib.request
import os, re, json, random
# 마르코프 체인 딕셔너리 만들기 --- (※1)
def make_dic(words):
    tmp = ["@"]
    dic = {}
    for word in words:
        tmp.append(word)
        if len(tmp) < 3: continue
        if len(tmp) > 3: tmp = tmp[1:]
        set_word3(dic, tmp)
        if word == ".":
            tmp = ["@"]
            continue
    return dic
# 딕셔너리에 데이터 등록하기 --- (※2)
def set_word3(dic, s3):
    w1, w2, w3 = s3
    if not w1 in dic: dic[w1] = {}
    if not w2 in dic[w1]: dic[w1][w2] = {}
    if not w3 in dic[w1][w2]: dic[w1][w2][w3] = 0
    dic[w1][w2][w3] += 1
# 문장 만들기 --- (※3)
def make_sentence(dic):
    ret = []
    if not "@" in dic: return "no dic" 
    top = dic["@"]
    w1 = word_choice(top)
    w2 = word_choice(top[w1])
    ret.append(w1)
    ret.append(w2)
    while True:
        w3 = word_choice(dic[w1][w2])
        ret.append(w3)
        if w3 == ".": break
        w1, w2 = w2, w3
    ret = "".join(ret)
    # 띄어쓰기
    params = urllib.parse.urlencode({
        "_callback": "",
        "q": ret
    })
    # 네이버 맞춤법 검사기를 사용합니다.
    data = urllib.request.urlopen("https://m.search.naver.com/p/csearch/dcontent/spellchecker.nhn?" + params)
    data = data.read().decode("utf-8")[1:-2]
    data = json.loads(data)
    data = data["message"]["result"]["html"]
    data = soup = BeautifulSoup(data, "html.parser").getText()
    # 리턴
    return data

def word_choice(sel):
    keys = sel.keys()
    return random.choice(list(keys))

# 문장 읽어 들이기 --- (※4)
toji_file = "toji.txt"
dict_file = "markov-toji.json"
if not os.path.exists(dict_file):
    # 토지 텍스트 파일 읽어 들이기
    fp = codecs.open("BEXX0003.txt", "r", encoding="utf-16")
    soup = BeautifulSoup(fp, "html.parser")
    body = soup.select_one("body > text")
    text = body.getText()
    text = text.replace("…", "") # 현재 koNLPy가 …을 구두점으로 잡지 못하는 문제 임시 해결
    # 형태소 분석
    twitter = Twitter()
    malist = twitter.pos(text, norm=True)
    words = []
    for word in malist:
        # 구두점 등은 대상에서 제외(단 마침표는 포함)
        if not word[1] in ["Punctuation"]:
            words.append(word[0])
        if word[0] == ".":
            words.append(word[0])
    # 딕셔너리 생성
    dic = make_dic(words)
    json.dump(dic, open(dict_file,"w", encoding="utf-8"))
else:
    dic = json.load(open(dict_file,"r"))
# 문장 만들기 --- (※6)
for i in range(3):
    s = make_sentence(dic)
    print(s)
    print("---")

헤 헤 입을 초병마개처럼 내가 물씬 코를 찔렀다.
---
저녁은 안 먹어 안 묵소 떡도 않는데 부엉이는 무척 가까운 곳에서는 장작불이 훨훨 타오르고있었인께좀점잖아지겄소삼월이젖은눈에심이세기로범겉은종놈들은 곰방대를 팽개치고 전복을 입고 관대를 두르고 나갔다.
---
디뚝디뚝걷다가용이는여전히음산했다뿐인가지금으로 치면 정승 벼슬까지 하셨는데 이리 빈둥거리고 있어 쓰겠나 야.
---
